In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Авторизация гугл диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [4]:
# создаем датасет для проверки
assert_values = [
    [0, 708930000.0, 688998322.0, 666349089.0, 647380821.0, 630300887.0, 611463914.0, 591781038.0],
    [1, 721780000.0, 700858040.0, 679984420.0, 661238265.0, 644054439.0, 624842853.0, 604260080.0],
    [2, 676197680.0, 657479234.0, 635803739.0, 616560408.0, 599282032.0, 582739733.0, 564951055.0],
    [3, 705906000.0, 684776146.0, 664699620.0, 646675310.0, 629384170.0, 611508787.0, 590382699.0],
    [4, 697650000.0, 677569923.0, 657200106.0, 638066185.0, 620003731.0, 600610165.0, 583305516.0],
    [5, 690886000.0, 670980106.0, 650141712.0, 631199828.0, 613500459.0, 596658378.0, 578937537.0],
    [6, 725780000.0, 703148787.0, 681992058.0, 661459515.0, 641162461.0, 621924929.0, 602339667.0],
]
assert_data = pd.DataFrame(assert_values, columns=['MONTHS_PAST_DEFAULT'] + [f'DEBT_{i}' for i in range(7)])
assert_data = assert_data.set_index('MONTHS_PAST_DEFAULT')

# Загрузка данных

Наша стартовая таблица "Dataset_UP_LGD.csv" со следующими полями:
* ID - идентификационный номер клиента 
* DEBT_0 - сумма долга клиента в месяц дефолта
* PAYMENT_0	- сумма платежа клиента в месяц дефолта
* PAYMENT_1	- сумма платежа клиента в следующем месяце после дефолта клиента
* PAYMENT_2	- сумма платежа клиента через 2 месяца после дефолта клиента	
* PAYMENT_3	- сумма платежа клиента через 3 месяца после дефолта 	
* PAYMENT_4	- сумма платежа клиента через 4 месяца после дефолта 	
* PAYMENT_5	- сумма платежа клиента через 5 месяца после дефолта 	
* PAYMENT_6	- сумма платежа клиента через 6 месяца после дефолта 

In [5]:
data = pd.read_csv('./Dataset_UP_LGD.csv')
data.head()

,ID,DEBT_0,PAYMENT_0,PAYMENT_1,PAYMENT_2,PAYMENT_3,PAYMENT_4,PAYMENT_5,PAYMENT_6
0,1,20000.0,0,0.0,689.0,0.0,0.0,0.0,0.0
1,2,120000.0,0,0.0,1000.0,1000.0,NaN,NaN,NaN
2,3,90000.0,0,1518.0,1500.0,1000.0,1000.0,NaN,NaN
3,4,50000.0,0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0
4,6,50000.0,0,2500.0,1815.0,657.0,1000.0,NaN,NaN


# Task 1. Вычисление долга

Напишите функцию, которая возвращает по каждому клиенту информацию о его долге в каждый месяц после дефолта.

Функция на вход принимает таблицу data

Результатом работы функции является DataFrame с полями:
* ID - идентификационный номер клиента
* MONTHS_PAST_DEFAULT - количество месяцев после дефолта (количество столбцов с долгом != Nan - 1)
* DEBT_0 - сумма долга в момент дефолта
* DEBT_1 - сумма долга в следующем месяце после дефолта
* DEBT_2 - сумма долга через 2 есяца после дефолта
* DEBT_3 - сумма долга через 3 есяца после дефолта
* DEBT_4 - сумма долга через 4 есяца после дефолта
* DEBT_5 - сумма долга через 5 есяца после дефолта
* DEBT_6 - сумма долга через 6 есяца после дефолта

In [6]:
def calc_debt(data_):
    df = {}
    for id in data_['ID'].unique():
        debt0 = data_[data_['ID'] == id]['DEBT_0'].values[0]
        debt1 = debt0 - data_[data_['ID'] == id]['PAYMENT_1'].values[0]
        debt2 = debt1 - data_[data_['ID'] == id]['PAYMENT_2'].values[0]
        debt3 = debt2 - data_[data_['ID'] == id]['PAYMENT_3'].values[0]
        debt4 = debt3 - data_[data_['ID'] == id]['PAYMENT_4'].values[0]
        debt5 = debt4 - data_[data_['ID'] == id]['PAYMENT_5'].values[0]
        debt6 = debt5 - data_[data_['ID'] == id]['PAYMENT_6'].values[0]
        df[id] =  {'ID' : id, 'MONTHS_PAST_DEFAULT' : 6 - data_[data_['ID'] == id].isna().sum().sum(), 'DEBT_0' : debt0, 'DEBT_1' : debt1, 'DEBT_2' : debt2, 'DEBT_3' : debt3, 'DEBT_4' : debt4, 'DEBT_5' : debt5, 'DEBT_6' : debt6}
    return pd.DataFrame(df).T

In [7]:
data = calc_debt(data_ = data.copy())

assert data.shape == (29034, 9)
assert data.sum().sum() == 19019524231.0
assert data.count().sum() == 174319

# Task 2. Группировка наблюдений

Реалиуйте функцию, которая возвращает общую сумму долга для каждой группы наблюдений с одинаковым возрастом дефолта в каждый месяц после дефолта.

Функция на вход принимает таблицу с долгом data

Результатом работы функции является DataFrame с полями:

* MONTHS_PAST_DEFAULT - количество месяцев после дефолта в качестве ИНДЕКСА
* DEBT_0 - сумма долга в момент дефолта
* DEBT_1 - сумма долга в следующем месяце после дефолта
* DEBT_2 - сумма долга через 2 есяца после дефолта
* DEBT_3 - сумма долга через 3 есяца после дефолта
* DEBT_4 - сумма долга через 4 есяца после дефолта
* DEBT_5 - сумма долга через 5 есяца после дефолта
* DEBT_6 - сумма долга через 6 есяца после дефолта

Учтите следующее: для тех месяцев, по которым нет данных о долге в таблице должно быть проставлено NaN

In [8]:
def group_debt(data_):
    return data_.groupby('MONTHS_PAST_DEFAULT').sum()[['DEBT_0', 'DEBT_1', 'DEBT_2', 'DEBT_3','DEBT_4', 'DEBT_5', 'DEBT_6']].replace(0, np.NaN)

In [9]:
data_gr = group_debt(data_ = data.copy())

assert data_gr.shape == (7, 7)
assert data_gr.sum().sum() == 18584769614.0
assert data_gr.count().sum() == 28

In [10]:
data_gr

,DEBT_0,DEBT_1,DEBT_2,DEBT_3,DEBT_4,DEBT_5,DEBT_6
MONTHS_PAST_DEFAULT,,,,,,,
0.0,708930000.0,NaN,NaN,NaN,NaN,NaN,NaN
1.0,721780000.0,700858040.0,NaN,NaN,NaN,NaN,NaN
2.0,676197680.0,657479234.0,635803739.0,NaN,NaN,NaN,NaN
3.0,705906000.0,684776146.0,664699620.0,646675310.0,NaN,NaN,NaN
4.0,697650000.0,677569923.0,657200106.0,638066185.0,620003731.0,NaN,NaN
5.0,690886000.0,670980106.0,650141712.0,631199828.0,613500459.0,596658378.0,NaN
6.0,725780000.0,703148787.0,681992058.0,661459515.0,641162461.0,621924929.0,602339667.0


# Task 3. Вычисление коэффициентов

Реализуйте функцию, которая возвращает список коэффициентов переходов для суммы долга между суммами долга в разные месяцы.

Функция на вход принимает сгруппированную таблицу data_gr

Результатом работы функции является np.array с коэффициентами перехода (округлите коэффициенты перехода до 5 знака).

In [ ]:
def calc_coefs(data_):
    coefs = []
    raise NotImplementedError
    # return np.round(coefs, 5)

In [ ]:
coefs = calc_coefs(data_ = data_gr)
print(coefs)
assert coefs.shape == (6,)
assert round(coefs.sum(), 5) == 5.82191

# Task 4. Заполнение пропусков

Реалиуйте функцию, которая заполняет пропуски в сгруппированной таблице долга.

Функция на вход принимает:
* таблицу со сгруппированным долгом data_gr
* список коэффициентов coefs

Результатом работы функции является заполненный DataFrame с полями:

* MONTHS_PAST_DEFAULT - количество месяцев после дефолта в качестве ИНДЕКСА
* DEBT_0 - сумма долга в момент дефолта
* DEBT_1 - сумма долга в следующем месяце после дефолта
* DEBT_2 - сумма долга через 2 есяца после дефолта
* DEBT_3 - сумма долга через 3 есяца после дефолта
* DEBT_4 - сумма долга через 4 есяца после дефолта
* DEBT_5 - сумма долга через 5 есяца после дефолта
* DEBT_6 - сумма долга через 6 есяца после дефолта


In [ ]:
def fill_data(data_, coefs):
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
data_gr_filled = fill_data(data_ = data_gr.copy(),
                           coefs = np.round(coefs, 5)
                           )

assert data_gr_filled.shape == (7, 7)
assert round(data_gr_filled.sum().sum(), 5) == 31578683486.38918
assert data_gr_filled.count().sum() == 49

In [ ]:
data_gr_filled

Средняя абсолютная ошибка долга по заполненным месяцам

In [ ]:
abs((data_gr_filled - assert_data) / assert_data).sum().sum()/((data_gr_filled.shape[0]*data_gr_filled.shape[1] - data_gr_filled.shape[0])/2)

Теперь можно рассчитать ожидаемый уровень LGD на горизонте 6 месяцев

In [ ]:
(data_gr_filled['DEBT_6'] / data_gr_filled['DEBT_0']).mean()